# Riot Games ML Models - Model Registry

This notebook trains the three ML models required for the Riot Games Intelligence Agent:
- **Revenue Forecasting** - Predict future monthly revenue from player purchases
- **Player Churn Risk Classification** - Identify players likely to quit the game
- **Toxicity Prediction** - Estimate the probability that a player will have behavior incidents

All models are registered to Snowflake Model Registry and exposed to the agent through `sql/ml/07_create_model_wrapper_functions.sql`.

## Prerequisites

**Required Packages** (configured automatically):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database Context:**
- **Database:** RIOT_GAMES_INTELLIGENCE  
- **Schema:** ANALYTICS  
- **Warehouse:** RIOT_GAMES_SI_WH

**Note:** This notebook uses Snowflake Model Registry. Ensure you have appropriate permissions to create and register models.


## Import Required Packages


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")

## Connect to Snowflake

Get the active session and set context to the Riot Games database.


In [ ]:
session = get_active_session()

session.use_database('RIOT_GAMES_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('RIOT_GAMES_SI_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")

---
# MODEL 1: Revenue Forecasting

Predict future monthly revenue using historical purchase activity.


### Prepare Revenue Training Data


In [ ]:
revenue_df = session.sql("""
SELECT
    DATE_TRUNC('month', purchase_date)::DATE AS purchase_month,
    MONTH(purchase_date) AS month_num,
    YEAR(purchase_date) AS year_num,
    SUM(amount_usd)::FLOAT AS total_revenue,
    COUNT(DISTINCT player_id)::FLOAT AS player_count,
    AVG(amount_usd)::FLOAT AS avg_transaction_amount,
    COUNT(DISTINCT CASE WHEN item_type = 'SKIN' THEN transaction_id END)::FLOAT AS skin_purchase_count
FROM RAW.PURCHASES
WHERE amount_usd > 0
  AND purchase_date >= DATEADD('month', -36, CURRENT_DATE())
GROUP BY 1,2,3
ORDER BY purchase_month
""")

print(f"Revenue data: {revenue_df.count()} months")
revenue_df.show(5)

### Split Data and Train Revenue Model


In [ ]:
train_revenue = revenue_df.filter(F.col("PURCHASE_MONTH") < F.dateadd("month", F.lit(-6), F.current_date()))
test_revenue = revenue_df.filter(F.col("PURCHASE_MONTH") >= F.dateadd("month", F.lit(-6), F.current_date()))

train_revenue = train_revenue.drop("PURCHASE_MONTH")
test_revenue = test_revenue.drop("PURCHASE_MONTH")

revenue_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "PLAYER_COUNT", "AVG_TRANSACTION_AMOUNT", "SKIN_PURCHASE_COUNT"],
        output_cols=["MONTH_NUM_SCALED", "PLAYER_COUNT_SCALED", "AVG_TRANSACTION_SCALED", "SKIN_COUNT_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["TOTAL_REVENUE"],
        output_cols=["PREDICTED_REVENUE"]
    ))
])

revenue_pipeline.fit(train_revenue)
print("✅ Revenue forecasting model trained")

### Evaluate and Register Revenue Model


In [ ]:
test_predictions = revenue_pipeline.predict(test_revenue)

mae = mean_absolute_error(df=test_predictions, y_true_col_names="TOTAL_REVENUE", y_pred_col_names="PREDICTED_REVENUE")
mse = mean_squared_error(df=test_predictions, y_true_col_names="TOTAL_REVENUE", y_pred_col_names="PREDICTED_REVENUE")
rmse = mse ** 0.5

metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {metrics}")

reg = Registry(session)
reg.log_model(
    model=revenue_pipeline,
    model_name="REVENUE_FORECASTER",
    version_name="V1",
    comment="Predicts monthly revenue from player purchases using Linear Regression",
    metrics=metrics
)

print("✅ Revenue model registered to Model Registry as REVENUE_FORECASTER")

---
# MODEL 2: Player Churn Risk Classification

Classify players as likely to churn based on engagement and spending patterns.


### Prepare Churn Training Data


In [ ]:
churn_df = session.sql("""
SELECT
    p.player_id,
    p.player_segment AS segment,
    p.ranked_tier AS rank,
    p.account_level::FLOAT AS account_level,
    p.honor_level::FLOAT AS honor_level,
    p.total_playtime_hours::FLOAT AS total_playtime_hours,
    COUNT(DISTINCT m.match_id)::FLOAT AS total_matches,
    COALESCE(SUM(pur.amount_usd), 0)::FLOAT AS lifetime_spending,
    DATEDIFF('day', MAX(m.match_date), CURRENT_DATE())::FLOAT AS days_since_last_match,
    COUNT_IF(m.afk_flag)::FLOAT AS afk_count,
    (DATEDIFF('day', MAX(m.match_date), CURRENT_DATE()) > 30)::BOOLEAN AS is_churned
FROM RAW.PLAYERS p
LEFT JOIN RAW.MATCH_HISTORY m ON p.player_id = m.player_id
LEFT JOIN RAW.PURCHASES pur ON p.player_id = pur.player_id
GROUP BY 1,2,3,4,5,6
HAVING COUNT(DISTINCT m.match_id) > 0
LIMIT 10000
""")

print(f"Churn data: {churn_df.count()} players")
churn_df.show(5)

### Train Churn Classification Model


In [ ]:
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)

train_churn = train_churn.drop("PLAYER_ID")
test_churn = test_churn.drop("PLAYER_ID")

churn_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["SEGMENT", "RANK"],
        output_cols=["SEGMENT_ENCODED", "RANK_ENCODED"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_CHURNED"],
        output_cols=["CHURN_PREDICTION"],
        n_estimators=200,
        max_depth=12
    ))
])

churn_pipeline.fit(train_churn)
print("✅ Churn risk classification model trained")

### Evaluate and Register Churn Model


In [ ]:
churn_predictions = churn_pipeline.predict(test_churn)

accuracy = accuracy_score(df=churn_predictions, y_true_col_names="IS_CHURNED", y_pred_col_names="CHURN_PREDICTION")
churn_metrics = {"accuracy": round(accuracy, 4)}
print(f"Churn model metrics: {churn_metrics}")

reg.log_model(
    model=churn_pipeline,
    model_name="CHURN_RISK_MODEL",
    version_name="V1",
    comment="Predicts player churn risk using Random Forest on engagement patterns",
    metrics=churn_metrics
)

print("✅ Churn risk model registered to Model Registry as CHURN_RISK_MODEL")

---
# MODEL 3: Toxicity Prediction

Predict the probability that a player will have behavior incidents.


### Prepare Toxicity Training Data


In [ ]:
toxicity_df = session.sql("""
SELECT
    p.player_id,
    p.player_segment AS segment,
    p.ranked_tier AS rank,
    p.honor_level::FLOAT AS honor_level,
    COUNT(DISTINCT m.match_id)::FLOAT AS total_matches,
    COUNT_IF(m.afk_flag)::FLOAT AS afk_count,
    AVG(m.deaths)::INT AS avg_deaths,
    COUNT(DISTINCT ir.incident_report_id)::FLOAT AS past_incidents,
    DATEDIFF('day', p.account_created_date, CURRENT_DATE())::FLOAT AS account_age_days,
    (COUNT(DISTINCT ir.incident_report_id) > 0)::BOOLEAN AS has_incident
FROM RAW.PLAYERS p
LEFT JOIN RAW.MATCH_HISTORY m ON p.player_id = m.player_id
LEFT JOIN RAW.INCIDENT_REPORTS ir ON p.player_id = ir.player_id
GROUP BY 1,2,3,4,9
HAVING COUNT(DISTINCT m.match_id) > 0
LIMIT 10000
""")

print(f"Toxicity data: {toxicity_df.count()} players")
toxicity_df.show(5)

### Train Toxicity Prediction Model


In [ ]:
train_toxicity, test_toxicity = toxicity_df.random_split([0.8, 0.2], seed=42)

train_toxicity = train_toxicity.drop("PLAYER_ID")
test_toxicity = test_toxicity.drop("PLAYER_ID")

toxicity_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["SEGMENT", "RANK"],
        output_cols=["SEGMENT_ENC", "RANK_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["HAS_INCIDENT"],
        output_cols=["TOXICITY_PREDICTION"]
    ))
])

toxicity_pipeline.fit(train_toxicity)
print("✅ Toxicity prediction model trained")

### Evaluate and Register Toxicity Model


In [ ]:
toxicity_predictions = toxicity_pipeline.predict(test_toxicity)

toxicity_accuracy = accuracy_score(
    df=toxicity_predictions,
    y_true_col_names="HAS_INCIDENT",
    y_pred_col_names="TOXICITY_PREDICTION"
)
toxicity_metrics = {"accuracy": round(toxicity_accuracy, 4)}
print(f"Toxicity model metrics: {toxicity_metrics}")

reg.log_model(
    model=toxicity_pipeline,
    model_name="TOXICITY_PREDICTION_MODEL",
    version_name="V1",
    comment="Predicts the likelihood of player behavior incidents",
    metrics=toxicity_metrics
)

print("✅ Toxicity model registered to Model Registry as TOXICITY_PREDICTION_MODEL")

---
# Verify Models in Registry


In [ ]:
print("Models in registry:")
reg.show_models()

print("\nRevenue Forecaster versions:")
reg.get_model("REVENUE_FORECASTER").show_versions()

print("\nChurn Risk Model versions:")
reg.get_model("CHURN_RISK_MODEL").show_versions()

print("\nToxicity Prediction Model versions:")
reg.get_model("TOXICITY_PREDICTION_MODEL").show_versions()

print("\n✅ All models registered and ready to add to the Intelligence Agent")

---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
revenue_model = reg.get_model("REVENUE_FORECASTER").default
recent_revenue = revenue_df.limit(3).drop("PURCHASE_MONTH")
revenue_preds = revenue_model.run(recent_revenue, function_name="predict")
print("Revenue predictions:")
revenue_preds.select("TOTAL_REVENUE", "PREDICTED_REVENUE").show()

churn_model = reg.get_model("CHURN_RISK_MODEL").default
sample_churn = churn_df.limit(5).drop("PLAYER_ID")
churn_preds = churn_model.run(sample_churn, function_name="predict")
print("\nChurn risk predictions:")
churn_preds.select("IS_CHURNED", "CHURN_PREDICTION").show()

toxicity_model = reg.get_model("TOXICITY_PREDICTION_MODEL").default
sample_toxicity = toxicity_df.limit(5).drop("PLAYER_ID")
toxicity_preds = toxicity_model.run(sample_toxicity, function_name="predict")
print("\nToxicity predictions:")
toxicity_preds.select("HAS_INCIDENT", "TOXICITY_PREDICTION").show()

print("\n✅ All models tested successfully!")

---
# Next Steps

## Add Models to Intelligence Agent

**Run the SQL Script:**
Execute `sql/agent/08_create_intelligence_agent.sql` which automatically configures all 3 ML models.

**Manual Configuration (Alternative):**
1. In Snowsight → AI & ML → Agents → RIOT_GAMES_INTELLIGENCE_AGENT
2. Go to Tools → + Add → Function
3. Add each model wrapper procedure:
   - **PREDICT_REVENUE** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_CHURN_RISK** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_TOXICITY_RISK** (from `sql/ml/07_create_model_wrapper_functions.sql`)

## Example Questions for Agent

- "Predict revenue for the next 6 months"
- "Which CASUAL players are at high churn risk?"
- "What is the toxicity risk for ENGAGED segment players?"
- "Show me revenue trends by player segment and predict next quarter"

The models will now be available as tools your agent can use!
